In [24]:
classifier = pipeline("sentiment-analysis")

In [32]:
from transformers.pipelines import get_supported_tasks

In [30]:
get_supported_tasks()

['audio-classification',
 'automatic-speech-recognition',
 'conversational',
 'feature-extraction',
 'fill-mask',
 'image-classification',
 'image-segmentation',
 'ner',
 'object-detection',
 'question-answering',
 'sentiment-analysis',
 'summarization',
 'table-question-answering',
 'text-classification',
 'text-generation',
 'text2text-generation',
 'token-classification',
 'translation',
 'zero-shot-classification',
 'zero-shot-image-classification']

In [17]:
import transformers
import pandas as pd
classifier = pipeline("sentiment-analysis")
ner_tagger = pipeline("ner", aggregation_strategy="simple", model="")
qa = pipeline("question-answering", model="")
translator = pipeline("translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de")

transformers.logging.set_verbosity_error()

<h1 style="text-align:center;">A Whirlwind Tour of the 🤗 Hugging Face Ecosystem</h1>

<br><br><br><br>

<h3 style="text-align:center;"><b>Christopher Akiki</b></h3>

<center><img src="images/chapter01_hf-ecosystem.png" width=800></center>

<center><img src="images/chapter02_hf-libraries.png" width=1800></center>

<h1 style="text-align:center;">🤗 Pipelines</h1>

<br><br>

In [2]:
from transformers import pipeline


<center><img src="images/gewandhaus_review.png" width=900></center>

In [3]:
text = """One of the best orchestra in the world. I came to Leipzig\
            mainly to have one experience with Gewanhaus Leipzig Orchestra. 
            Under the baton of Maestro Andris Nelsons, Bruckner symphony #8 was so affection. 
            The acustic and layout of the concert hall is nice."""

# Sentiment Analysis

```python
classifier = pipeline("text-classification")
```

In [19]:
outputs = classifier(text)
outputs[0]

{'label': 'POSITIVE', 'score': 0.9998534917831421}

# Named-Entity Recognition

```python
ner_tagger = pipeline("ner", aggregation_strategy="simple")
```

In [20]:
outputs = ner_tagger(text)
pd.DataFrame(outputs)

,entity_group,score,word,start,end
0,LOC,0.999257,Leipzig,50,57
1,ORG,0.990783,Gewanhaus Leipzig Orchestra,104,131
2,PER,0.996171,Andris Nelsons,173,187
3,MISC,0.564720,B,189,190
4,ORG,0.268703,##ck,192,194
5,MISC,0.364942,##ner,194,197


# Question Answering

```python
qa = pipeline("question-answering")
```

In [21]:
question = "Why did I visit Leipzig?"
outputs = qa(question=question, context=text)
outputs

{'score': 0.5873121023178101,
 'start': 76,
 'end': 131,
 'answer': 'to have one experience with Gewanhaus Leipzig Orchestra'}

In [22]:
question = "What music did the orchestra play?"
outputs = qa(question=question, context=text)
outputs

{'score': 0.1337369829416275,
 'start': 189,
 'end': 209,
 'answer': 'Bruckner symphony #8'}

# Translation

```python
translator = pipeline("translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de")
```

In [18]:
outputs = translator(text, clean_up_tokenization_spaces=True)
print(outputs[0]['translation_text'])

Ich bin vor allem nach Leipzig gekommen, um eine Erfahrung mit dem Gewanhaus Leipzig Orchestra zu machen. Unter der Leitung von Maestro Andris Nelsons war die Bruckner Sinfonie #8 so liebevoll, dass die Akustik und Gestaltung des Konzertsaales schön ist.


<h1 style="text-align:center;">🤗 Tokenizers</h1>

<center><img src="images/tokenization_pipeline.svg" width=1200></center>

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('gutenberg')
nltk.download('punkt')

In [ ]:
print(nltk.corpus.gutenberg.fileids())

In [ ]:
moby_dick_raw = nltk.corpus.gutenberg.raw('melville-moby_dick.txt')
moby_dick_sentences = sent_tokenize(moby_dick, language='english')

In [ ]:
len(moby_dick_sentences)

In [ ]:
from tokenizers import Tokenizer, normalizers, pre_tokenizers, processors
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer

In [ ]:
unk_token = "[UNK]"
pad_token = "[PAD]"
cls_token = "[CLS]" 
sep_token = "[SEP]"
mask_token = "[MASK]"
special_tokens = [unk_token, pad_token, cls_token, sep_token, mask_token]
vocab_size = 20_000

In [ ]:
custom_tokenizer = Tokenizer(WordPiece(unk_token=unk_token))

In [ ]:
custom_normalizer = normalizers.Sequence(
            [normalizers.NFKD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

In [ ]:
custom_pre_tokenizer = pre_tokenizers.Sequence(
            [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
)

In [ ]:
custom_trainer = WordPieceTrainer(vocab_size=vocab_size, special_tokens=special_tokens, show_progress=False)

In [ ]:
custom_tokenizer.normalizer = custom_normalizer
custom_tokenizer.pre_tokenizer = custom_pre_tokenizer

In [ ]:
custom_tokenizer.train_from_iterator(moby_dick_sentences, trainer=custom_trainer)

In [ ]:
custom_tokenizer.get_vocab_size()

In [ ]:
encoding = custom_tokenizer.encode("Let us test this tokenizer")
print(encoding.tokens)

In [ ]:
cls_token_id = tokenizer.token_to_id(cls_token)
sep_token_id = tokenizer.token_to_id(sep_token)

custom_post_processor = processors.TemplateProcessing(
    single=f"{cls_token}:0 $A:0 {sep_token}:0",
    pair=f"{cls_token}:0 $A:0 {sep_token}:0 $B:1 {sep_token}:1",
    special_tokens=[(cls_token, cls_token_id), (sep_token, sep_token_id)],
)

custom_tokenizer.post_processor = custom_post_processor

In [ ]:
encoding = custom_tokenizer.encode("Let us test this tokenizer")
print(encoding.tokens)

In [ ]:
encoding = custom_tokenizer.encode("This is the first sentence", "This is sentence number 2")
print(encoding.tokens)
print(encoding.ids)
print(encoding.type_ids)

# Using our custom tokenizer with 🤗 Transformers

In [ ]:
from transformers import PreTrainedTokenizerFast

PreTrainedTokenizerFast


<h1 style="text-align:center;">🤗 Datasets</h1>

<table><thead><tr><th align="center">Data format</th> <th align="center">Loading script</th> <th align="center">Example</th></tr></thead> <tbody><tr><td align="center">CSV &amp; TSV</td> <td align="center"><code>csv</code></td> <td align="center"><code>load_dataset("csv", data_files="my_file.csv")</code></td></tr> <tr><td align="center">Text files</td> <td align="center"><code>text</code></td> <td align="center"><code>load_dataset("text", data_files="my_file.txt")</code></td></tr> <tr><td align="center">JSON &amp; JSON Lines</td> <td align="center"><code>json</code></td> <td align="center"><code>load_dataset("json", data_files="my_file.jsonl")</code></td></tr> <tr><td align="center">Pickled DataFrames</td> <td align="center"><code>pandas</code></td> <td align="center"><code>load_dataset("pandas", data_files="my_dataframe.pkl")</code></td></tr></tbody></table>

<h1 style="text-align:center;">🤗 Transformers</h1>

<h1 style="text-align:center;">Case-study: 📜 Scientific Paper Retrieval</h1>

<h1 style="text-align:center;">(Re)sources</h1>

- https://github.com/nlp-with-transformers/notebooks

- https://github.com/huggingface/course


<center><a href="https://www.oreilly.com/library/view/natural-language-processing/9781098103231/"><img src="images/book_cover.png" width=500></a></center>